# Get AgERA5 data

This notebook allows for downloading of daily AgERA5 climate data from the [Copernicus Climate Data Store (CDS)](https://cds.climate.copernicus.eu/#!/home), and preprocessing for their direct use in crop simulation models of the SARRA family. The output format is a series of geotiff files, one per variable and per day.

The cdsapi package is used to download data from CDS. This has the advantage of magaging the caching of already passed requests, thus to speed up the downloading process.

AgERA5 data should be produced daily, with a 7-day lag, according to the [AgERA-5 frequency update](https://confluence.ecmwf.int/display/CUSF/AgERA-5+frequency+update) documentation.

Before running the notebook for the first time, [follow these instructions](https://cds.climate.copernicus.eu/api-how-to) to setup a Copernicus Climate Data Store API key on your machine, and accept [Copernicus Terms of Service](https://cds.climate.copernicus.eu/cdsapp/#!/terms/licence-to-use-copernicus-products).

The output files should be stored in the './data/3_output/' directory.

## 1. Imports

In [1]:
import sys
sys.path.append('../src/SARRA_data_download/')
from get_AgERA5_data import *

## 2. Parameters

In [2]:
# defining dictionary of coordinates for areas of interest
# bounding box coordinates format : [lat NW, lon NW, lat SE, lon SE]
area = {
    'burkina': [16, -6, 9, 3],
    'niger':[23.8, -0.5, 11.3, 15.9],
    'west_africa':[29, -20, 3.5, 26]}

# selecting area of interest
selected_area = "burkina"

# selecting year of interest
query_year = 2020

# defining save path
save_path = "../data/"

## 3. Run

In [3]:
download_AgERA5_year(query_year, area, selected_area, save_path)

===== download_AgERA5_data =====
Mode 'year' acknowledged. Will download data for year 2020
Please note that last available date on AgERA5 should be 2023-03-22


2023-03-30 15:30:34,767 INFO Welcome to the CDS
2023-03-30 15:30:34,768 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agrometeorological-indicators
2023-03-30 15:30:34,854 INFO Request is completed
2023-03-30 15:30:34,855 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data9/dataset-sis-agrometeorological-indicators-ab6c914a-9fd0-437f-8a79-4e1b02b6b20c.zip to ../data/0_downloads/AgERA5_burkina_2m_temperature_24_hour_minimum_2020.zip (13.1M)
2023-03-30 15:30:37,278 INFO Download rate 5.4M/s   
2023-03-30 15:30:37,359 INFO Welcome to the CDS
2023-03-30 15:30:37,360 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agrometeorological-indicators
2023-03-30 15:30:37,450 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data5/dataset-sis-agrometeorological-indicators-b64e763e-de43-4737-94be-5bae40aac98d.zip to ../data/0_downloads/AgERA5_burkina_2m_temperatur

Download OK


2023-03-30 15:30:40,721 INFO Download rate 4M/s     
2023-03-30 15:30:40,809 INFO Welcome to the CDS
2023-03-30 15:30:40,810 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agrometeorological-indicators
2023-03-30 15:30:40,901 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data4/dataset-sis-agrometeorological-indicators-073c7857-35b6-426a-886e-fa8702e0012f.zip to ../data/0_downloads/AgERA5_burkina_solar_radiation_flux_daily_2020.zip (13.9M)


Download OK


...

Extraction OK
===== convert_AgERA5_netcdf_to_geotiff =====


100%|██████████| 366/366 [00:18<00:00, 19.58it/s]


===== calculate_AgERA5_ET0_and_save =====


100%|██████████| 366/366 [00:45<00:00,  8.10it/s]

===== Query date 2020-01-01 all done ! =====
